#### Importing Libraries:

In [1]:
import os, sys
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
import random
import pickle

In [100]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

#### Defining Targets:

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
with open('/content/drive/My Drive/train.csv', 'r') as f:
  df = pd.read_csv(f)
df

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0,0,1,0,0,0,0,0,1,1,0,5b5b9ac30
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,0,1,0,7192404d8
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0,0,1,0,0,1,0,1,0,1,0,d4d1b066d
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0,0,0,0,0,0,0,0,1,0,0,01a6602b8


In [25]:
#The data at hand includes:

with open('/content/drive/My Drive/my_data_2.csv', 'r') as f: ##my_data.csv
  my_df = pd.read_csv(f, header = None) ##oddly it has 12 extra NaN columns, but that is fine.
my_df

,0
0,1.2.826.0.1.3680043.8.498.58418707772781824332...
1,1.2.826.0.1.3680043.8.498.58419921389739675789...
2,1.2.826.0.1.3680043.8.498.58423010438223156361...
3,1.2.826.0.1.3680043.8.498.58431397580060014231...
4,1.2.826.0.1.3680043.8.498.58436636344879710159...
...,...
5406,1.2.826.0.1.3680043.8.498.79862437756023054051...
5407,1.2.826.0.1.3680043.8.498.79868214196606941038...
5408,1.2.826.0.1.3680043.8.498.79874285141876207487...
5409,1.2.826.0.1.3680043.8.498.79877223599278501218...


In [29]:
#deleting the extention ".jpg" from "my_data":

for i in range(0, len(my_df)):
    my_df[0][i] = my_df[0][i][:-4]

In [33]:
#finding the matched data:

temp_arr = df['StudyInstanceUID'].values.tolist()
lst_index = []

for i in range(0, len(my_df)):
    lst_index.append(temp_arr.index(my_df[0][i]))
    
# #A slower method of O(n^2):    
# for i in range(0, len(my_df)): #or using .find() method
#     for j in range(0, len(df)):
#         if my_df[0][0] == df['StudyInstanceUID'][j]:
#             lst_index.append(j)    
    
lst_index.sort()
# lst_index

In [34]:
#either deletes the excess data from the original dataset, or sorts them.

labels = df.iloc[lst_index, :]
labels

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
13,1.2.826.0.1.3680043.8.498.71008408169501434503...,0,0,0,0,0,0,0,0,0,1,0,7d2c4c9e3
17,1.2.826.0.1.3680043.8.498.58764130686095794423...,0,0,0,0,0,0,0,0,0,1,0,d1a2e61aa
26,1.2.826.0.1.3680043.8.498.65953940794722178092...,0,0,0,0,0,0,0,0,0,1,0,43991f139
40,1.2.826.0.1.3680043.8.498.66351209110039920496...,0,0,0,0,0,0,0,0,0,1,0,621ef014c
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30062,1.2.826.0.1.3680043.8.498.69652950520868382438...,0,0,0,0,0,0,0,0,0,1,0,56f75914e
30069,1.2.826.0.1.3680043.8.498.67810475638153448553...,0,0,1,0,0,0,1,0,0,0,0,bc1d59b76
30074,1.2.826.0.1.3680043.8.498.60620865844062547094...,0,0,0,0,0,0,0,0,1,0,0,45d82b916
30076,1.2.826.0.1.3680043.8.498.59704742952729813362...,0,0,0,0,0,0,0,1,0,0,0,172c3c7ed


In [35]:
#solving data leackage issue:

temp_arr = labels['PatientID'].values.tolist()
temp_lst = []
lst_id = []

for i in range(0, len(labels)):
    for j in range(i+1, len(labels)):
        if temp_arr[i] == temp_arr[j]:
            temp_lst.append(j) #labels.index[j]
    if temp_lst != []:
        temp_lst.insert(0, i) #labels.index[i]
        lst_id.append(temp_lst)
        temp_lst = []

# len(lst_id) #verified!
# lst_id
arr_id = np.array(lst_id)
arr_id

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


array([list([1, 68, 530, 568, 1110, 1753, 1877, 2209, 2732, 3466, 4020, 4854, 4947, 5377]),
       list([3, 837, 2524, 2676, 3301]),
       list([5, 408, 425, 713, 841, 864, 1240, 1485, 2402, 2940, 2975, 3369, 4027, 4305, 4626, 5410]),
       ..., list([5339, 5344, 5372]), list([5344, 5372]),
       list([5359, 5378])], dtype=object)

In [36]:
#dropping unnecessary columns in the df "labels":

labels = labels.drop(['StudyInstanceUID', 'PatientID'], axis=1)
# labels

In [37]:
arr_labels = labels.values
arr_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 1, 0]])

In [38]:
arr_labels.shape

(5411, 11)

#### Resizing images:

In [ ]:
# !pwd
os.chdir("./drive/My Drive/train/dataset")

In [105]:
numSamples = len(arr_labels) ##1000  ##10000
def load_resize_training_data(imageSize = 224, numSamples = numSamples): #all image sizes: 650*650 #also changed from 10000 to 1000 due to my system memory.
    path = !pwd
    path = path[0] 
    trainingData = []
    i=0
    for img in os.listdir():
        if i == numSamples:
            break
        img_path = path + '/' + img
        img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) #plt.imshow will not result in a very fair respresentation.
        resized_array = cv2.resize(img_array, (imageSize, imageSize))
        trainingData.append(resized_array) #train_df.at[i, 'combined_label']]
        i += 1

    return trainingData

data_train = load_resize_training_data()
data_train

[array([[  0,   0,   0, ...,  11,  10,  10],
        [  0,   0,   0, ...,   9,   9,   9],
        [  0,   0,   0, ...,   9,   9,   9],
        ...,
        [  0,   0,   0, ..., 126, 136, 142],
        [  0,   0,   0, ..., 156, 161, 171],
        [  0,   0,   0, ..., 176, 177, 185]], dtype=uint8),
 array([[  1,   1,   1, ...,   0,   1,   2],
        [ 16,  13,  13, ..., 117, 117, 124],
        [ 14,  12,  11, ...,  53,  53,  58],
        ...,
        [ 28,  18,  16, ...,  32,  22,  12],
        [ 21,  17,  17, ...,  33,  22,  11],
        [ 19,  14,  14, ...,  32,  23,   9]], dtype=uint8),
 array([[ 0,  0, 86, ..., 13, 13, 13],
        [ 0,  0, 96, ..., 12, 12, 12],
        [ 0,  0, 85, ..., 12, 12, 12],
        ...,
        [ 0,  0,  9, ..., 65, 68, 70],
        [ 0,  0,  6, ..., 68, 68, 71],
        [ 0,  0,  7, ..., 63, 67, 69]], dtype=uint8),
 array([[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
       

In [106]:
len(data_train) #verified!

5411

In [107]:
data_train[0].shape #verified!

(224, 224)

In [108]:
arr_train = np.array(data_train)

In [109]:
arr_train.shape

(5411, 224, 224)

In [110]:
#So far, we have the x-ray images in the "arr_train" variable. -> saving to "train.pickle"
#And also the 11 labels in the "arr_labels" variable. -> saving to "target.pickle"
#And all the repeted PatientIDs in the "arr_id" variable. -> saving to "id.pickle"

#### Saving the files:

In [111]:
#save x data
pickle_out = open("train.pickle", "wb")
pickle.dump(arr_train, pickle_out)
pickle_out.close()
#save y data
pickle_out = open("target.pickle", "wb")
pickle.dump(arr_labels, pickle_out)
pickle_out.close()
#save PatientIds:
pickle_out = open("id.pickle", "wb")
pickle.dump(arr_id, pickle_out)
pickle_out.close()

In [119]:
from google.colab import files

files.download("train.pickle")
files.download("target.pickle")
files.download("id.pickle")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>